In [1]:
import sys
import numpy as np
import pandas as pd
import os 
import gc
import math
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder
import datetime
import time
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import BaggingRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [11]:
path  = 'D:/MyDocument/Project/CarSalesPrediction/'

train_sales  = pd.read_csv(path+'train2_dataset/train_sales_data.csv')
train_search = pd.read_csv(path+'train2_dataset/train_search_data.csv')
train_user_reply = pd.read_csv(path+'train2_dataset/train_user_reply_data.csv')
evaluation_public = pd.read_csv(path+'test2_dataset/evaluation_public.csv')
data = pd.concat([train_sales, evaluation_public], ignore_index=True)
data = data.merge(train_search, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user_reply, 'left', on=['model', 'regYear', 'regMonth'])
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
data['bodyType'] = data['model'].map(train_sales.drop_duplicates('model').set_index('model')['bodyType'])

for i in ['bodyType', 'model']:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))
data['mt'] = (data['regYear']-2016)*12+data['regMonth']

In [3]:
data.loc[(data['regMonth']==1) & (data['regYear']==2016) & (data['model']==1) & (data['adcode'] != 110000)][['salesVolume','adcode']]

,salesVolume,adcode
22,389.0,310000
23,306.0,530000
24,260.0,150000
26,757.0,510000
27,275.0,340000
28,890.0,370000
29,387.0,140000
30,8330.0,440000
31,801.0,450000
32,1685.0,320000


In [4]:
data.loc[0]['province']

'上海'

In [5]:
# def get_stat_feature(df_):   
#     df = df_.copy()
#     stat_feat = []
#     y = []
#     df['model_adcode'] = df['adcode'] + df['model']
#     df['model_adcode_mt'] = df['model_adcode'] * 100 + df['mt']
#     for col in tqdm(['salesVolume','popularity','carCommentVolum','newsReplyVolum',\
#                'adcode_year_month_sales_all','adcode_year_month_bodyType_sales_all','model_year_month_sales_all']):
#         # shift
#         for i in [1,2,3,4,5,6,7,8,9,10,11,12]:
#             stat_feat.append('shift_model_adcode_mt_{}_{}'.format(col,i))
#             df['model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'] + i
#             df_last = df[~df[col].isnull()].set_index('model_adcode_mt_{}_{}'.format(col,i))
#             df['shift_model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'].map(df_last[col])
    
#     for col in tqdm(['salesVolume','popularity','carCommentVolum','newsReplyVolum',\
#                'adcode_year_month_sales_all','adcode_year_month_bodyType_sales_all','model_year_month_sales_all']):
#         # shift
#         for pair in [(1,2), (1,3), (2,3), (2,4), (3,4), (3,5), (1,6), (2,6), (1,12)]:
#             stat_feat.append('rise_model_adcode_mt_{}_{}_{}'.format(col,pair[0],pair[1]))
#             df['rise_model_adcode_mt_{}_{}_{}'.format(col,pair[0],pair[1])] = df['shift_model_adcode_mt_{}_{}'.format(col,pair[0])] - df['shift_model_adcode_mt_{}_{}'.format(col,pair[1])]

#     for col in tqdm(['salesVolume', 'adcode_year_month_sales_all','adcode_year_month_bodyType_sales_all','model_year_month_sales_all']):
#         # history feature delta
#         for pair in [(1,2), (1,3), (2,3), (2,4), (3,4), (3,5), (1,6), (1,12)]:
#             stat_feat.append('ratio_model_adcode_mt_{}_{}_{}'.format(col,pair[0],pair[1]))
#             df['ratio_model_adcode_mt_{}_{}_{}'.format(col,pair[0],pair[1])] = df['shift_model_adcode_mt_{}_{}'.format(col,pair[0])] \
#                                                / (df['shift_model_adcode_mt_{}_{}'.format(col,pair[1])])

#     for col in tqdm(['salesVolume']):
#         for i in [0,1,2,3]:
#             y.append('after_model_adcode_mt_{}_{}'.format(col,i))
#             df['model_adcode_mt_{}_{}'.format(col,-i)] = df['model_adcode_mt'] - i
#             df_last = df[~df[col].isnull()].set_index('model_adcode_mt_{}_{}'.format(col,-i))
#             df['after_model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'].map(df_last[col])
#     return df,stat_feat,y

In [18]:
def get_stat_feature(df_,): 
    df = df_.copy()
    stat_feat = []
    stat_feat_2=[]
    stat_feat_3 = []
    stat_feat_4 = []
    df['model_adcode'] = df['adcode'] + df['model']
    df['model_adcode_mt'] = df['model_adcode'] * 100 + df['mt']
    for col in ['label']:
        # 历史销量数据特征
        for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]:
            stat_feat.append('shift_model_adcode_mt_{}_{}'.format(col,i))
            df['model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'] + i
            df_last = df[~df[col].isnull()].set_index('model_adcode_mt_{}_{}'.format(col,i))
            df['shift_model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'].map(df_last[col])
    for col in ['popularity','carCommentVolum','newsReplyVolum']:
        # 历史popularity数据特征
        for i in [5,10,11,12]:# popularity只取一部分
            stat_feat.append('shift_model_adcode_mt_{}_{}'.format(col,i))
            df['model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'] + i
            df_last = df[~df[col].isnull()].set_index('model_adcode_mt_{}_{}'.format(col,i))
            df['shift_model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'].map(df_last[col])
    
    pairs = [(1,13), (2,14), (3,15), (4,16)]
    for col in ['label']:
        # 比去年同期增长多少(同比一年前的增长)
        for pair in pairs:
            stat_feat.append('increase_{}_{}_{}'.format(col,pair[0],pair[1]))
            df['increase_{}_{}_{}'.format(col,pair[0],pair[1])] = (df['shift_model_adcode_mt_{}_{}'.format(col,pair[0])]-
                df['shift_model_adcode_mt_{}_{}'.format(col,pair[1])])/df['shift_model_adcode_mt_{}_{}'.format(col,pair[1])]
        # 过去一段时间的mean, max, median, min, std, diff
        for width in [(1,4), (1,7), (1,5), (7,13), (3,6), (3,9)]:
            item = []
            for ii in range(width[0],width[1]):
                item.append('shift_model_adcode_mt_{}_{}'.format(col,ii))
            df['diff_{}_{}'.format(col,width)] = df[item].diff(axis=1).mean(axis=1)
            stat_feat.append('diff_{}_{}'.format(col,width))
            df['mean_{}_{}'.format(col,width)] = df[item].mean(axis=1)
            stat_feat.append('mean_{}_{}'.format(col,width))
            df['max_{}_{}'.format(col,width)] = df[item].max(axis=1)
            stat_feat.append('max_{}_{}'.format(col,width))
            df['min_{}_{}'.format(col,width)] = df[item].min(axis=1)
            stat_feat.append('min_{}_{}'.format(col,width))
            df['std_{}_{}'.format(col,width)] = df[item].std(axis=1)
            stat_feat.append('std_{}_{}'.format(col,width))
            df['median_{}_{}'.format(col,width)] = df[item].median(axis=1)
            stat_feat.append('median_{}_{}'.format(col,width))

        # 按车型或省份每月mean和min
        for ind in [1,2,3,4,5,6,7,8,9,10,11,12]:
            stat_feat.append('Mean_model_{}_{}'.format(col,ind))
            stat_feat.append('Min_model_{}_{}'.format(col,ind))
            stat_feat.append('Median_model_{}_{}'.format(col,ind))
            stat_feat.append('Max_model_{}_{}'.format(col,ind))
            stat_feat.append('Std_model_{}_{}'.format(col,ind))
            #  stat_feat.append('Diff_model_{}_{}'.format(col,ind))
            
            mean = pd.DataFrame(df.groupby(['model','mt'])['shift_model_adcode_mt_{}_{}'.format(col,ind)].mean()).rename(
                columns={'shift_model_adcode_mt_{}_{}'.format(col,ind):'Mean_model_{}_{}'.format(col,ind)})
            df = pd.merge(df,mean,on=["model","mt"],how="left")
            min_ = pd.DataFrame(df.groupby(['model','mt'])['shift_model_adcode_mt_{}_{}'.format(col,ind)].min()).rename(
                columns={'shift_model_adcode_mt_{}_{}'.format(col,ind):'Min_model_{}_{}'.format(col,ind)})
            df = pd.merge(df,min_,on=["model","mt"],how="left")
            max_ = pd.DataFrame(df.groupby(['model','mt'])['shift_model_adcode_mt_{}_{}'.format(col,ind)].max()).rename(
                columns={'shift_model_adcode_mt_{}_{}'.format(col,ind):'Max_model_{}_{}'.format(col,ind)})
            df = pd.merge(df,max_,on=["model","mt"],how="left")
            median_ = pd.DataFrame(df.groupby(['model','mt'])['shift_model_adcode_mt_{}_{}'.format(col,ind)].median()).rename(
                columns={'shift_model_adcode_mt_{}_{}'.format(col,ind):'Median_model_{}_{}'.format(col,ind)})
            df = pd.merge(df,median_,on=["model","mt"],how="left")
            std_ = pd.DataFrame(df.groupby(['model','mt'])['shift_model_adcode_mt_{}_{}'.format(col,ind)].std()).rename(
                columns={'shift_model_adcode_mt_{}_{}'.format(col,ind):'Std_model_{}_{}'.format(col,ind)})
            df = pd.merge(df,std_,on=["model","mt"],how="left")
            #  diff_ = pd.DataFrame(df.groupby(['model','mt'])['shift_model_adcode_mt_{}_{}'.format(col,ind)].diff().mean()).rename(
            #      columns={'shift_model_adcode_mt_{}_{}'.format(col,ind):'Diff_model_{}_{}'.format(col,ind)})
            #  df = pd.merge(df,diff_,on=["model","mt"],how="left")
            
            stat_feat.append('Mean_adcode_{}_{}'.format(col,ind))
            stat_feat.append('Min_adcode_{}_{}'.format(col,ind))
            stat_feat.append('Median_adcode_{}_{}'.format(col,ind))
            stat_feat.append('Max_adcode_{}_{}'.format(col,ind))
            stat_feat.append('Std_adcode_{}_{}'.format(col,ind))
            #  stat_feat.append('Diff_adcode_{}_{}'.format(col,ind))
            mean = pd.DataFrame(df.groupby(['adcode','mt'])['shift_model_adcode_mt_{}_{}'.format(col,ind)].mean()).rename(
                columns={'shift_model_adcode_mt_{}_{}'.format(col,ind):'Mean_adcode_{}_{}'.format(col,ind)})
            df = pd.merge(df,mean,on=["adcode","mt"],how="left")
            min_ = pd.DataFrame(df.groupby(['adcode','mt'])['shift_model_adcode_mt_{}_{}'.format(col,ind)].min()).rename(
                columns={'shift_model_adcode_mt_{}_{}'.format(col,ind):'Min_adcode_{}_{}'.format(col,ind)})
            df = pd.merge(df,min_,on=["adcode","mt"],how="left")
            max_ = pd.DataFrame(df.groupby(['adcode','mt'])['shift_model_adcode_mt_{}_{}'.format(col,ind)].max()).rename(
                columns={'shift_model_adcode_mt_{}_{}'.format(col,ind):'Max_adcode_{}_{}'.format(col,ind)})
            df = pd.merge(df,max_,on=["adcode","mt"],how="left")
            median_ = pd.DataFrame(df.groupby(['adcode','mt'])['shift_model_adcode_mt_{}_{}'.format(col,ind)].median()).rename(
                columns={'shift_model_adcode_mt_{}_{}'.format(col,ind):'Median_adcode_{}_{}'.format(col,ind)})
            df = pd.merge(df,median_,on=["adcode","mt"],how="left")
            std_ = pd.DataFrame(df.groupby(['adcode','mt'])['shift_model_adcode_mt_{}_{}'.format(col,ind)].std()).rename(
                columns={'shift_model_adcode_mt_{}_{}'.format(col,ind):'Std_adcode_{}_{}'.format(col,ind)})
            df = pd.merge(df,std_,on=["adcode","mt"],how="left")
            #  diff_ = pd.DataFrame(df.groupby(['adcode','mt'])['shift_model_adcode_mt_{}_{}'.format(col,ind)].diff().mean()).rename(
            #     columns={'shift_model_adcode_mt_{}_{}'.format(col,ind):'Diff_adcode_{}_{}'.format(col,ind)})
            #  df = pd.merge(df,diff_,on=["adcode","mt"],how="left")
    pairs = [(1,4), (1,6), (1,12), (3,6), (3,12),(3,9)]
    for col in ['label']:
        # mean model/adcode 增长比例
        for pair in pairs:
            stat_feat.append('increase_mean_model_{}_{}_{}'.format(col,pair[0],pair[1]))
            df['increase_mean_model_{}_{}_{}'.format(col,pair[0],pair[1])] = (df['Mean_model_{}_{}'.format(col,pair[0])]-
                df['Mean_model_{}_{}'.format(col,pair[1])])/df['Mean_model_{}_{}'.format(col,pair[1])]
            stat_feat.append('increase_mean_adcode_{}_{}_{}'.format(col,pair[0],pair[1]))
            df['increase_mean_adcode_{}_{}_{}'.format(col,pair[0],pair[1])] = (df['Mean_adcode_{}_{}'.format(col,pair[0])]-
                df['Mean_adcode_{}_{}'.format(col,pair[1])])/df['Mean_adcode_{}_{}'.format(col,pair[1])]
    
    for col in ['label']:
        # sum求和
        ind = 0
        for pair in [(1,2,11,12),(1,2,3),(1,12)]:
            stat_feat.append('sum_{}_{}'.format(col,ind))
            df['sum_{}_{}'.format(col,ind)] = 0
            length = len(pair)
            for i in range(length):
                df['sum_{}_{}'.format(col,ind)] += df["shift_model_adcode_mt_{}_{}".format(col,pair[i])].values
            ind += 1
            
    # 删除特征
    #  stat_feat.remove("shift_model_adcode_mt_label_15")
    y =[]
    for col in tqdm(['salesVolume']):
        for i in [0,1,2,3]:
            y.append('after_model_adcode_mt_{}_{}'.format(col,i))
            df['model_adcode_mt_{}_{}'.format(col,-i)] = df['model_adcode_mt'] - i
            df_last = df[~df[col].isnull()].set_index('model_adcode_mt_{}_{}'.format(col,-i))
            df['after_model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'].map(df_last[col])
    return df,stat_feat,y

In [13]:
def score(data, pred='pred_label', label='label', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
#     data['label'] = np.expm1(data['label'].values)
#     data['pred_label'] = np.expm1(data['pred_label'].values)
#     data['label'] = np.expm1(data['label'].values)
    data_agg = data.groupby('model').agg({
        pred:  list,
        label: [list, 'mean']
    }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)

In [14]:
def get_model_type(train_x,train_y,valid_x,valid_y,m_type='lgb'):   
    if m_type == 'lgb':
        model = lgb.LGBMRegressor(
                                num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='rmse',
                                max_depth=-1, learning_rate=0.008, min_child_samples=5, random_state=2019,
                                n_estimators=4000, subsample=0.9, colsample_bytree=0.7,
                                )
        #  model = BaggingRegressor(model, n_estimators=8, random_state=0, n_jobs=1, max_samples=0.9,)
        #  model.fit(train_x, train_y)
        
        model.fit(train_x, train_y, 
                      eval_set=[(train_x, train_y),(valid_x, valid_y)], 
                      categorical_feature=cate_feat, 
                      early_stopping_rounds=100, verbose=100)
    elif m_type == 'xgb':
        model = xgb.XGBRegressor(
                                max_depth=5 , learning_rate=0.05, n_estimators=2000, 
                                objective='reg:gamma', tree_method = 'hist',subsample=0.9, 
                                colsample_bytree=0.7, min_child_samples=5,eval_metric = 'rmse' 
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              early_stopping_rounds=100, verbose=100)   
    
    return model

In [15]:
def get_train_model(df_, m, y, m_type='lgb'):
    df = df_.copy()
    # 数据集划分
    st = 13
    all_idx   = (df['mt'].between(st , 21))
    train_idx = (df['mt'].between(st , 20))
    valid_idx = (df['mt'].between(21, 21))
    test_idx  = (df['mt'].between(25, 25))

    train_x = df[train_idx][features]
    train_y = df[train_idx][y]
    valid_x = df[valid_idx][features]
    valid_y = df[valid_idx][y]   
    # get model
    model = get_model_type(train_x,train_y,valid_x,valid_y,m_type)  
    # offline
    df['pred_label'] = model.predict(df[features])
    # df['pred_label'] = np.expm1(df['pred_label'].values)
    best_score = score(df[valid_idx],label=y) 
    # online
    if m_type == 'lgb':
        model.n_estimators = model.best_iteration_+100
        model.fit(df[all_idx][features], df[all_idx][y])
    elif m_type == 'xgb':
        model.n_estimators = model.best_iteration+100
        model.fit(df[all_idx][features], df[all_idx][y])
    df['forecastVolum'] = model.predict(df[features]) 
    # df['forecastVolum'] = np.expm1(df['forecastVolum'].values)
    print('valid mean:',df[valid_idx]['pred_label'].mean())
    print('true  mean:',df[valid_idx][y].mean())
    print('test  mean:',df[test_idx]['forecastVolum'].mean())
    # 阶段结果
    sub = df[test_idx][['id']]
    sub['forecastVolum'] = df[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)  
    return sub,df[valid_idx]['pred_label']

In [19]:
data_df, stat_feat, y = get_stat_feature(data)
m_type = 'lgb' 
    
num_feat = stat_feat+['regYear']
cate_feat = ['adcode','bodyType','model','regMonth']
    
if m_type == 'lgb':
    for i in cate_feat:
        data_df[i] = data_df[i].astype('category')
elif m_type == 'xgb':
    lbl = LabelEncoder()  
    for i in tqdm(cate_feat):
        data_df[i] = lbl.fit_transform(data_df[i].astype(str))
features = num_feat + cate_feat
print(len(features), len(set(features))) 

for month in [0,1,2,3]: 
    sub,val_pred = get_train_model(data_df, month, y[month], m_type)   
    data.loc[(data.regMonth==(month+1))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month+1))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
# sub['forecastVolum'] = np.expm1(sub['forecastVolum'].values)
sub[['id','forecastVolum']].round().astype(int).to_csv('2_4_new_lgb.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.11it/s]


208 208
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 303.676	valid_1's rmse: 387.869
[200]	training's rmse: 174.214	valid_1's rmse: 235.694
[300]	training's rmse: 120.526	valid_1's rmse: 178.53
[400]	training's rmse: 96.811	valid_1's rmse: 160.466
[500]	training's rmse: 84.0229	valid_1's rmse: 153.77
[600]	training's rmse: 75.456	valid_1's rmse: 150.417
[700]	training's rmse: 68.9314	valid_1's rmse: 148.776
[800]	training's rmse: 64.2168	valid_1's rmse: 147.499
[900]	training's rmse: 60.343	valid_1's rmse: 146.907
[1000]	training's rmse: 57.1868	valid_1's rmse: 146.607
[1100]	training's rmse: 54.5083	valid_1's rmse: 146.316
[1200]	training's rmse: 52.1953	valid_1's rmse: 146.14
[1300]	training's rmse: 50.2049	valid_1's rmse: 145.956
[1400]	training's rmse: 48.4904	valid_1's rmse: 145.934
[1500]	training's rmse: 46.9	valid_1's rmse: 145.827
[1600]	training's rmse: 45.4314	valid_1's rmse: 145.78
[1700]	training's rmse: 44.095	valid_1's rmse: 145.68